# IMPORT

In [16]:
import os, json, glob, pickle
import numpy as np
from tqdm import tqdm
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# PROSES LOOK VECTOR

In [17]:
def parse_look_vec(vec_str):
    # Input "(-0.2425, -0.0858, -0.9663, 0.0000)"
    vec_str = vec_str.replace("(", "").replace(")", "")
    parts = vec_str.split(",")
    x = float(parts[0])
    y = float(parts[1])
    return x, y   # only need x, y

# LABEL MAPPING

In [18]:
def gaze_to_label(gx, gy, thr=0.12):
    if gx < -thr: return "left"
    if gx >  thr: return "right"
    if gy < -thr: return "up"
    if gy >  thr: return "down"
    return "center"


# LOAD DATASETS

In [19]:
DATA_DIR = "../../data/eye-datasets" 
json_files = sorted(glob.glob(os.path.join(DATA_DIR, "*.json")))
print("Total JSON:", len(json_files))

features = []
labels = []

for jf in tqdm(json_files):
    with open(jf, "r") as f:
        ann = json.load(f)

    # target field: eye_details → look_vec
    look = ann["eye_details"]["look_vec"]
    
    gx, gy = parse_look_vec(look)
    label = gaze_to_label(gx, gy)

    features.append([gx, gy])
    labels.append(label)

print("Loaded samples:", len(features))

Total JSON: 3149


  0%|          | 0/3149 [00:00<?, ?it/s]

100%|██████████| 3149/3149 [00:00<00:00, 13088.69it/s]

Loaded samples: 3149


In [20]:
X = np.array(features)
y = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model = MLPClassifier(
    hidden_layer_sizes=(32, 16),
    activation='relu',
    max_iter=500
)

model.fit(X_train, y_train)
pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, pred))
print(classification_report(y_test, pred))

Accuracy: 0.9968253968253968
              precision    recall  f1-score   support

      center       0.96      0.96      0.96        26
        down       1.00      0.98      0.99        46
        left       1.00      1.00      1.00       257
       right       1.00      1.00      1.00       255
          up       1.00      1.00      1.00        46

    accuracy                           1.00       630
   macro avg       0.99      0.99      0.99       630
weighted avg       1.00      1.00      1.00       630



In [22]:
with open("unityeyes_eye_model.pkl", "wb") as f:
    pickle.dump(model, f)

print("Saved model → unityeyes_eye_model.pkl")


Saved model → unityeyes_eye_model.pkl
